In [17]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import pickle
from sklearn.metrics import classification_report
from sklearn.datasets import make_circles, make_moons
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
with open('cifar/cifar-100-python/train', 'rb') as f:
    data_train = pickle.load(f, encoding='latin1')
with open('cifar/cifar-100-python/test', 'rb') as f:
    data_test = pickle.load(f, encoding='latin1')

# Здесь указать ваши классы по варианту!!!
# 1. Номер группы + 15
# 2. Номер варианта + 56
# 3. ИУ5 (Номер варианта + 21); ГУИМЦ (80); Иностранцы (90)
# (39 — клавиатуры, 58 — небо, 23 — внедорожники)
CLASSES = [39, 58, 23]

# массив
train_X = data_train['data'].reshape(-1, 3, 32, 32)
train_X = np.transpose(train_X, [0, 2, 3, 1]) # NCHW -> NHWC
train_y = np.array(data_train['fine_labels'])
mask = np.isin(train_y, CLASSES) # фильтруем данные, выбираем только те, которые относятся к нашим классам

train_X = train_X[mask].copy()
train_y = train_y[mask].copy()
train_y = np.unique(train_y, return_inverse=1)[1]
del data_train

test_X = data_test['data'].reshape(-1, 3, 32, 32)
test_X = np.transpose(test_X, [0, 2, 3, 1])
test_y = np.array(data_test['fine_labels'])
mask = np.isin(test_y, CLASSES)

test_X = test_X[mask].copy()
test_y = test_y[mask].copy()
test_y = np.unique(test_y, return_inverse=1)[1]
del data_test

In [19]:
class Normalize(nn.Module):
    
    def __init__(self, mean, std):
        super(Normalize, self).__init__()
        self.mean = torch.tensor(mean)
        self.std = torch.tensor(std)

    def forward(self, input):
        x = input / 255.0
        x = x - self.mean
        x = x / self.std
        return torch.flatten(x, start_dim=1) # nhwc -> nm

class Cifar100_MLP(nn.Module):
    def __init__(self, _seq : nn.Sequential):
        super(Cifar100_MLP, self).__init__()
        # https://blog.jovian.ai/image-classification-of-cifar100-dataset-using-pytorch-8b7145242df1
        self.norm = Normalize([0.5074,0.4867,0.4411],[0.2011,0.1987,0.2025])
        self.seq = _seq
        #self.seq = nn.Sequential(
        #    nn.Linear(32*32*3, hidden_size),
        #    nn.ReLU(),
        #    nn.Linear(hidden_size, classes),
        #)

    def forward(self, input):
        x = self.norm(input)
        return self.seq(x)

In [23]:
def task_3(_batch_size, _epochs, _classes: int, _learning_rate:float, seq: nn.Sequential):
  batch_size = _batch_size
  dataloader = {}

  for (X, y), part in zip([(train_X, train_y), (test_X, test_y)], ['train', 'test']):
      tensor_x = torch.Tensor(X)
      tensor_y = F.one_hot(torch.Tensor(y).to(torch.int64), num_classes=_classes)/1.
      dataset = TensorDataset(tensor_x, tensor_y)
      dataloader[part] = DataLoader(dataset, batch_size=batch_size, shuffle=True)

  model = Cifar100_MLP(seq)

  print(model)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=_learning_rate)

  EPOCHS = _epochs
  # train data set
  steps_per_epoch_train = len(dataloader['train'])
  # test data set (валидация)
  steps_per_epoch_test = len(dataloader['test'])

  losses_train_x = []
  losses_train_y = []
  losses_test_x = []
  losses_test_y = []

  for epoch in range(EPOCHS):  # проход по набору данных несколько раз
      running_loss = 0.0
      model.train()

      for i, batch in enumerate(dataloader['train'], 0):
          # получение одного минибатча; batch это двуэлементный список из [inputs, labels]
          inputs, labels = batch

          # очищение прошлых градиентов с прошлой итерации
          optimizer.zero_grad()

          # прямой + обратный проходы + оптимизация
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          
          loss.backward()
          optimizer.step()

          # для подсчёта статистик
          running_loss += loss.item()

      # график
      losses_train_x.append(epoch)
      losses_train_y.append(running_loss / steps_per_epoch_train)

      running_loss = 0.0
      model.eval() # модель переводится в evaluation mode (режим оценки)
      with torch.no_grad(): # отключение автоматического дифференцирования
          for i, batch in enumerate(dataloader['test'], 0):
              inputs, labels = batch

              outputs = model(inputs)
              loss = criterion(outputs, labels)
              running_loss += loss.item()

      # график
      losses_test_x.append(epoch)
      losses_test_y.append(running_loss / steps_per_epoch_test)

  print('Обучение закончено')
  
  min_losses_test_y = max(losses_test_y)
  min_losses_test_x = 0
  for i in range(0, len(losses_test_y)):
      if losses_test_y[i] < min_losses_test_y:
          min_losses_test_y = losses_test_y[i]
          min_losses_test_x = i
  
  print("Переобучение произошло на эпохе: ", min_losses_test_x)
  print("Ошибка в момент переобучения: ", min_losses_test_y)

  for part in ['train', 'test']:
    y_pred = []
    y_true = []
    with torch.no_grad(): # отключение автоматического дифференцирования
        for i, data in enumerate(dataloader[part], 0):
            inputs, labels = data

            outputs = model(inputs).detach().numpy()
            y_pred.append(outputs)
            y_true.append(labels.numpy())
        y_true = np.concatenate(y_true)
        y_pred = np.concatenate(y_pred)
        print(part)
        print(classification_report(y_true.argmax(axis=-1), y_pred.argmax(axis=-1), digits=4, target_names=list(map(str, CLASSES))))
        print('-'*50)

In [34]:
BATCH_SIZE = 128
HIDDEN_SIZE = 32
LEARNING_RATE = 0.005
NUM_OF_CLASSES = 3
EPOCHS = 250
def default_sequention():
    return nn.Sequential(
            # полносвязный слой (входной - скрытый)
            nn.Linear(32*32*3, HIDDEN_SIZE),
            # функция активации для нейронов скрытого слоя
            nn.ReLU(),
            # полносвязный слой (скрытый - выходной)
            nn.Linear(HIDDEN_SIZE, NUM_OF_CLASSES),
        )

In [36]:
task_3(BATCH_SIZE, EPOCHS, NUM_OF_CLASSES, LEARNING_RATE, default_sequention())

Cifar100_MLP(
  (norm): Normalize()
  (seq): Sequential(
    (0): Linear(in_features=3072, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=3, bias=True)
  )
)
Обучение закончено
Переобучение произошло на эпохе:  91
Ошибка в моменте переобучения:  0.5351938704649607
train
              precision    recall  f1-score   support

          39     0.9920    0.9900    0.9910       500
          58     0.9920    0.9920    0.9920       500
          23     0.9980    1.0000    0.9990       500

    accuracy                         0.9940      1500
   macro avg     0.9940    0.9940    0.9940      1500
weighted avg     0.9940    0.9940    0.9940      1500

--------------------------------------------------
test
              precision    recall  f1-score   support

          39     0.6967    0.8500    0.7658       100
          58     0.7442    0.6400    0.6882       100
          23     0.8696    0.8000    0.8333       100

    accuracy                    